In [2]:
import requests
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
from io import BytesIO
import PyPDF2

In [10]:

def parse_recipes(text):
    recipes = []
    recipe_blocks = text.split('\n\n')
    for block in recipe_blocks:
        if block.strip(): #skip empty blocks
            lines = block.split('\n')
            title =  'No Title'
            if not (title.lower().startswith('o') and title.lower().endswith('o')):
                title = 'No title found'
            ingredients = []
            instructions = []

            for line in lines[2:]: # lines staring from second line
                line = line.strip()
                if line:
                    if line.startswith('H'):
                        ingredients.append(line)
                    else:
                        instructions.append(line)

    ingredients_text = ' '.join(ingredients) if ingredients else 'No ingredients found'
    instructions_text = ' '.join(instructions) if instructions else 'No instructions found'

    recipes.append({'Title': title, 'Ingredients': ingredients_text, 'Instructions': instructions_text})
    return recipes

# Function to download PDF from URL
def download_pdf(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Check for request errors
    return BytesIO(response.content)  # Return as a BytesIO object

def extract_recipes(pdf_url, start_page, end_page, output_csv):
    pdf_file = download_pdf(pdf_url)

    pdf_reader = PyPDF2.PdfReader(pdf_file)

    all_recipes = []

    for page_num in range(start_page - 1, end_page):  # PyPDF2 uses 0-based indexing
        if page_num < len(pdf_reader.pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            if text:  # Check if text extraction was successful
                recipes = parse_recipes(text)
                all_recipes.extend(recipes)

    df = pd.DataFrame(all_recipes)
    df.to_csv(output_csv, index=False)

pdf_url = 'https://localfoodconnect.org.au/wp-content/uploads/2014/08/recipe-book.pdf'
start_page = 11
end_page = 151
output_csv = 'pick_share_eat.csv'

extract_recipes(pdf_url, start_page, end_page, output_csv)


In [9]:
def parse_recipes(text):
    recipes = []
    recipe_blocks = text.split('\n\n')
    for block in recipe_blocks:
        if block.strip():  # Skip empty blocks
            lines = block.split('\n')
            title = 'No Title'  # Default title
            ingredients = []
            instructions = []

            for line in lines:
                line = line.strip()
                if line:  # Skip empty lines
                    if line.lower().startswith('o') or line.upper().endswith('O') or line.lower().startswith('o ') or line.upper().endswith('  O'):
                        title = line  # Set the title if it meets the criteria
                    elif line.startswith('H'):  # Check if the line starts with 'H'
                        ingredients.append(line)
                    else:
                        instructions.append(line)

            ingredients_text = ' '.join(ingredients) if ingredients else 'No Ingredients'
            instructions_text = ' '.join(instructions) if instructions else 'No Instructions'
            
            recipes.append({
                'Title': title,
                'Ingredients': ingredients_text,
                'Instructions': instructions_text
            })
    return recipes

def download_pdf(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    response.raise_for_status()  
    return BytesIO(response.content)  

def extract_recipes(pdf_url, start_page, end_page, output_csv):
    pdf_file = download_pdf(pdf_url)

    pdf_reader = PyPDF2.PdfReader(pdf_file)

    all_recipes = []

    for page_num in range(start_page - 1, end_page):  # PyPDF2 uses 0-based indexing
        if page_num < len(pdf_reader.pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            if text:  
                recipes = parse_recipes(text)
                all_recipes.extend(recipes)

    df = pd.DataFrame(all_recipes)
    df.to_csv(output_csv, index=False)

pdf_url = 'https://localfoodconnect.org.au/wp-content/uploads/2014/08/recipe-book.pdf'
start_page = 11
end_page = 151
output_csv = 'pick_share_eat_1.csv'

extract_recipes(pdf_url, start_page, end_page, output_csv)


In [16]:
# Function to parse recipe text (customized to handle potential recipe structures)
def parse_recipes(text):
    recipes = []
    recipe_blocks = text.split('\n\n')
    
    for block in recipe_blocks:
        if block.strip():  # Skip empty blocks
            lines = block.split('\n')
            current_title = None
            ingredients = []
            instructions = []
            collecting_instructions = False  # Flag to indicate if we are collecting instructions
            
            for line in lines:
                line = line.strip()
                if line:  # Skip empty lines
                    if line.lower().startswith('o ') or line.upper().endswith('  O') and line.lower().startswith('o') or line.upper().endswith(' O') :
                        if current_title is not None:
                            print(current_title)
                            recipes.append({
                                'Title': current_title,
                                'Ingredients': ' '.join(ingredients) if ingredients else 'No Ingredients',
                                'Instructions': ' '.join(instructions) if instructions else 'No Instructions'
                            })

                            ingredients = []
                            instructions = []
                        
                        current_title = line
                        collecting_instructions = False  # Reset flag for new title
                    else:
                        if line.startswith('H'):
                            ingredients.append(line)
                        else:
                            instructions.append(line)
            
            if current_title is not None:
                recipes.append({
                    'Title': current_title,
                    'Ingredients': ' '.join(ingredients) if ingredients else 'No Ingredients',
                    'Instructions': ' '.join(instructions) if instructions else 'No Instructions'
                })
    
    return recipes

def download_pdf(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Check for request errors
    return BytesIO(response.content)  # Return as a BytesIO object

def extract_recipes(pdf_url, start_page, end_page, output_csv):
    pdf_file = download_pdf(pdf_url)

    pdf_reader = PyPDF2.PdfReader(pdf_file)

    all_recipes = []

    for page_num in range(start_page - 1, end_page):  # PyPDF2 uses 0-based indexing
        if page_num < len(pdf_reader.pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            if text:  # Check if text extraction was successful
                recipes = parse_recipes(text)
                all_recipes.extend(recipes)

    # Save recipes to CSV
    df = pd.DataFrame(all_recipes)
    df.to_csv(output_csv, index=False)

# Define parameters
pdf_url = 'https://localfoodconnect.org.au/wp-content/uploads/2014/08/recipe-book.pdf'
start_page = 11
end_page = 151
output_csv = 'pick_share_eat_2.csv'

# Run the extraction
extract_recipes(pdf_url, start_page, end_page, output_csv)


Springo Cooked Radishes  O
Spring o Leek & Parsley Soup  O
Springo Lemon Sponge  O
Spring o Meringue Eggs  O
Springo Mint Jelly  O
Spring o Pickled T ree Onions  O
o taste
o Silverbeet Patties  O
Spring o Spicy Carrot Dip O
o dust
Summer o Fresh Bean Salad  O
Summer o Green T omato Pickles  O
Summer o Hot Cucumber Soup  O
Summero Lilly Pilly Cordial  O
o 30mm pieces
o Zucchini Fritters  O
Summer o Zucchini Muffins  O
Autumno Green Babaco Salad  O
o 2 tomatoes, cut into long
Autumno Mushroom Risotto  O
Autumn o Pepino Sauce  O
Autumno Quince Crumble  O
Autumn o Pomegranate Syrup  O
Autumno Quince T arte T atin  O
Autumn o Red Salad  O
Autumn o Sautéed Kohlrabi  O
Autumn o Vegetable Pasties  O
Autumn o Yacon Syrup  O
Winter o Guava Jelly  O
Wintero Oca Saag Aloo  O
Winter o Snow Pea Salad  O
Wintero T amarillo Jam  O
Winter o T aro Curry  O
Wintero T uscan Kale Chips  O
